# Cell Optimization

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk

from apps.calcexamples.structure_browser import StructureBrowser


import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
structures = [("select structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms, node
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

## Step 1: Select Structure

In [ ]:
def on_struct_change(c):
    global atoms, node
    node = struct_browser.results.value
    if not node:
        return
    refresh_structure_view()
    cell_params.value = "Unit cell <br />a = {:.2f} {:.2f} {:.2f} <br />b = {:.2f} {:.2f} {:.2f} <br />c = {:.2f} {:.2f} {:.2f}".format(
        atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
        atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
        atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])

    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()
cell_params = ipw.HTML("Cell: ")
clear_output()
display(ipw.VBox([struct_browser, viewer, cell_params]))

## Step 2: Optimize the unit cell

In [ ]:
from aiida.orm import load_node, Code
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.utils import CalculationFactory
from aiida.work.run import submit

In [ ]:
def setup_calc():
    codename = 'cp2k_6.0_18265@daint-s746'
    code = Code.get_from_string(codename)
    options =  {'max_wallclock_seconds': 3600, 'resources': {'num_machines': 1}, 'queue_name':'normal'}


    parameters = {
        'GLOBAL': {
            'RUN_TYPE': 'CELL_OPT',
        },
        'FORCE_EVAL': {
            'METHOD': 'Quickstep',
            'DFT': {
                'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
                'POTENTIAL_FILE_NAME': 'POTENTIAL',
                'QS': {
                    'EPS_DEFAULT': 1.0e-12,
                    'WF_INTERPOLATION': 'ps',
                    'EXTRAPOLATION_ORDER': 3,
                },
                'MGRID': {
                    'NGRIDS': 4,
                    'CUTOFF': 280,
                    'REL_CUTOFF': 30,
                },
                'XC': {
                    'XC_FUNCTIONAL': {
                        '_': 'PBE',
                    },
                },
                'POISSON': {
                    'PERIODIC': 'none',
                    'PSOLVER': 'MT',
                },
            },
            'SUBSYS': {
                'KIND': [{'_': e , 'BASIS_SET': 'DZVP-MOLOPT-SR-GTH','POTENTIAL': 'GTH-PBE'} for e in node.get_kind_names()],
            },
        }
    }

    inputs = {
        'code': code,
        'structure': node,
        'parameters': ParameterData(dict=parameters),
        'settings': ParameterData(dict={}),
        '_options': options,
    }
    return inputs

In [ ]:
Cp2kCalculation = CalculationFactory('cp2k')

In [ ]:
def on_click_submit(b):
    with submit_out:
        process = Cp2kCalculation.process()
        calculation = submit(process, **setup_calc())
        print("Submitted calculation N {}".format(calculation.pid))

btn_submit = ipw.Button(description='Submit Cell Opt')
btn_submit.on_click(on_click_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)